## Sample

In [70]:
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "state-college-pa.osm"  # Replace this with your osm file
SAMPLE_FILE = "pa.osm"

k = 1000 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')



## Improving Street Names

In [149]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "state-college-pa.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Alley", "Box", "Building", "Center", "Circle", "Pike", "Way",
           '522']

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd.": "Road",
            "Aly": "Alley",
           "Blvd": "Boulevard",
           "Dr": "Drive",
           "Ln": "Lane",
           "Rd": "Road",
           "STreet": "Street",
            }

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    #pprint.pprint(dict(street_types))
    return street_types

def update_name(name, mapping):   
    m = street_type_re.search(name)
    if m not in expected:
        name = re.sub(m.group(), mapping[m.group()], name)
    return name

    
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
multiple_colons = re.compile(r':{2,}')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]


#The shape_element function runs on each element
def shape_element(element):
    node = {}
    address = {}
    if element.tag == "node" or element.tag == "way" :
        # created dictionary for creation info
        created = {}
        # pos list for lat and lon
        pos = []
        node['type'] = element.tag
        # loop through each element attribute
        for a in element.attrib.keys():
            # if key in CREATED list, add key:value to created dictionary
            if a in CREATED: 
                created[a] = element.attrib[a]
                # if created dictionary created, add to node dictionary
                if created: 
                    node['created'] = created
            # if key is lat or lon, add to pos list, then add list to node dict
            elif a == 'lat':
                pos.insert(0,element.get('lat'))
            elif a == 'lon':
                pos.insert(0,element.get('lon'))
                node['pos'] = pos
            # otherwise, add key:value pair of attribute to node dictionary
            else:
                node[a] = element.get(a)           
        for subtag in element:
            if subtag.get('k'):
                # if tag has two or more colons, ignore
                if re.search(r':.*:', subtag.get('k')):
                    continue
                #if tag has problem characters, ignore
                elif problemchars.search(subtag.get('k')):
                    continue
                #if tag is for street, check that the street name is in expected,
                # if not, change to the value in mapping dictionary
                elif subtag.get('k') == 'addr:street':
                    m = street_type_re.search(subtag.get('v'))
                    if m:
                        street_type = m.group()
                        if street_type not in expected:
                            address['street'] = update_name(subtag.get('v'), mapping)
                        else:
                            address['street'] = subtag.get('v')
                # add all other address values to address dictionary
                elif subtag.get('k').startswith('addr:'):
                    address[subtag.get('k')[5:]] = subtag.get('v')
                else:
                    node[subtag.get('k')] = subtag.get('v')
                if address:
                    node['address'] = address
        return node

    else:
        return None

def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        counter = 0
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
            # counter addeded to verify script is still running on larger datasets
            counter += 1
            if counter % 100000 == 0:
                print counter
    return data

def run():
    # NOTE: if you are running this code on your computer, with a larger dataset, 
    # call the process_map procedure with pretty=False. The pretty=True option adds 
    # additional spaces to the output, making it significantly larger.
    #data = process_map('C:\Users\Mister\Documents\data_analyst\P3_Data_Wrangling\example.osm', True)
    data = process_map('state-college-pa.osm', True)
if __name__ == "__main__":
    run()

100000
200000
300000
400000
500000
600000
700000
800000
900000


## Street names not in expected list

In [127]:
audit('state-college-pa.osm')

defaultdict(set,
            {'Aly': {'McAllister Aly'},
             'Ave': {'Delaware Ave',
              'E College Ave',
              'East Beaver Ave',
              'W College Ave',
              'W Freedom Ave',
              'West Freedom Ave'},
             'Blvd': {'Colonnade Blvd'},
             'Dr': {'Premiere Dr'},
             'Ln': {'Sandy Ln'},
             'Rd': {"McAlevy's Fort Rd"},
             'STreet': {'South Butz STreet'},
             'St': {'4th St',
              'E Main St',
              'Hiester St',
              'N Juniata St',
              'N Patterson St',
              'S Fraser St',
              'S Garner St',
              'S Hiester St',
              'S Sparks St'},
             'St.': {'North Atherton St.', 'S. Fraser St.'}})

## Import
1. In command prompt, cd to C:\Program Files\MongoDB\Server\3.2\bin.
2. mongo
3. db.pa.drop()
2. mongoimport -d test -c pa --file C:\Users\Mister\Documents\data_analyst\P3_Data_Wrangling\P3\state-college-pa.osm.json

In [65]:
from pymongo import MongoClient

client = MongoClient()
db = client.test

## Number of documents

In [66]:
db.pa.find().count()

348100

## DB Stats within MongoDB shell 

In command window:
1. mongo
2. show dbs
3. show collections
4. use test
5. coll = db.pa
6. coll.find().count()
7. coll.dataSize()
8. coll.stats()

(or 
db.pa.stats()
db.sample.dataSize())

## Number of nodes

In [151]:
db.pa.find({'type': 'node'}).count()

327544

## Number of ways

In [152]:
db.pa.find({'type': 'ways'}).count()

0

## Number of unique users

In [153]:
print len(db.pa.distinct("created.user"))

229


http://stackoverflow.com/questions/30327508/mongodb-osm-street-maps-unique-users
distinct returns a list.  to get the length of a list in python, use length()

## Top 5 users with highest number of entries

In [154]:
import pprint
cursor = db.pa.aggregate([
        {'$group': {'_id': "$created.user", 'count': {'$sum': 1}}},
        {'$sort': {'count': -1}},
        {'$limit': 5}
    ])
for doc in cursor:
    pprint.pprint(doc)

{u'_id': u'woodpeck_fixbot', u'count': 206010}
{u'_id': u'Sven L', u'count': 82402}
{u'_id': u'TIGERcnl', u'count': 8170}
{u'_id': u'bot-mode', u'count': 7491}
{u'_id': u'DaveHansenTiger', u'count': 4903}


## Number and Types of Buildings

In [155]:
import pprint
cursor = db.pa.aggregate([
        {'$group': {'_id': "$building", 'count': {'$sum': 1}}},
        {'$sort': {'count': -1}}
    ])
for doc in cursor:
    pprint.pprint(doc)

{u'_id': None, u'count': 347264}
{u'_id': u'yes', u'count': 716}
{u'_id': u'apartments', u'count': 57}
{u'_id': u'residential', u'count': 21}
{u'_id': u'entrance', u'count': 16}
{u'_id': u'house', u'count': 9}
{u'_id': u'commercial', u'count': 5}
{u'_id': u'retail', u'count': 4}
{u'_id': u'industrial', u'count': 2}
{u'_id': u'collapsed', u'count': 2}
{u'_id': u'ruins', u'count': 1}
{u'_id': u'public', u'count': 1}
{u'_id': u'office', u'count': 1}
{u'_id': u'Restaurant', u'count': 1}


## Postcode Check

In [158]:
print db.pa.distinct('address.postcode')

[u'16803', u'16801-9998', u'16801', u'16801-4736', u'16801-3838', u'16801-4922', u'16801-2812', u'16801-4713', u'16802-2604', u'16801-4032', u'16801-2810', u'16801-3923', u'16801-3922', u'16801-3919', u'16823', u'16669', u'16870', u'16801-7307', u'17004', u'16828', u'17841', u'16804', u'16858', u'16802', u'16801-2811', u'16801-3804', u'16827', u'16803-3073', u'16803-3044', u'16801-3896', u'17044', u'17009', u'PA', u'17747']


In [155]:
import pprint
cursor = db.pa.aggregate([
        {'$group': {'_id': "$building", 'count': {'$sum': 1}}},
        {'$sort': {'count': -1}}
    ])
for doc in cursor:
    pprint.pprint(doc)

{u'_id': None, u'count': 347264}
{u'_id': u'yes', u'count': 716}
{u'_id': u'apartments', u'count': 57}
{u'_id': u'residential', u'count': 21}
{u'_id': u'entrance', u'count': 16}
{u'_id': u'house', u'count': 9}
{u'_id': u'commercial', u'count': 5}
{u'_id': u'retail', u'count': 4}
{u'_id': u'industrial', u'count': 2}
{u'_id': u'collapsed', u'count': 2}
{u'_id': u'ruins', u'count': 1}
{u'_id': u'public', u'count': 1}
{u'_id': u'office', u'count': 1}
{u'_id': u'Restaurant', u'count': 1}


In [173]:
cursor = db.pa.aggregate([
        {'$group': {'_id': {'$substr': ['$address.postcode', 0, 5]}, 
                            'count': {'$sum': 1}}}])
for doc in cursor:
    pprint.pprint(doc)

{u'_id': u'16801', u'count': 273}
{u'_id': u'', u'count': 347772}
{u'_id': u'16828', u'count': 1}
{u'_id': u'16870', u'count': 1}
{u'_id': u'16804', u'count': 1}
{u'_id': u'16803', u'count': 26}
{u'_id': u'16669', u'count': 1}
{u'_id': u'16802', u'count': 7}
{u'_id': u'16823', u'count': 3}
{u'_id': u'17004', u'count': 1}
{u'_id': u'17841', u'count': 1}
{u'_id': u'16858', u'count': 5}
{u'_id': u'16827', u'count': 2}
{u'_id': u'17044', u'count': 2}
{u'_id': u'17009', u'count': 2}
{u'_id': u'PA', u'count': 1}
{u'_id': u'17747', u'count': 1}


## Type check

In [174]:
print db.pa.distinct('type')

[u'node', u'Public', u'way', u'route']


In [175]:
cursor = db.sample.find({'type': 'broad_leaved'})
for doc in cursor:
    print doc

{u'natural': u'tree', u'created': {u'changeset': u'35623297', u'version': u'4', u'user': u'woodpeck_repair', u'timestamp': u'2015-11-28T07:12:55Z', u'uid': u'145231'}, u'lon': u'-77.4423822', u'pos': [39.0586388, -77.4423822], u'_id': ObjectId('578d159c87b6d73f4147c126'), u'type': u'broad_leaved', u'id': u'2405872915'}
{u'natural': u'tree', u'created': {u'changeset': u'35623297', u'version': u'4', u'user': u'woodpeck_repair', u'timestamp': u'2015-11-28T07:12:55Z', u'uid': u'145231'}, u'pos': [u'39.0586388', u'-77.4423822'], u'_id': ObjectId('579a696de300de16b3497a08'), u'type': u'broad_leaved', u'id': u'2405872915'}
{u'natural': u'tree', u'created': {u'changeset': u'35623297', u'version': u'4', u'user': u'woodpeck_repair', u'timestamp': u'2015-11-28T07:12:55Z', u'uid': u'145231'}, u'pos': [u'39.0586388', u'-77.4423822'], u'_id': ObjectId('579e691e757d8f6e3fbdd01b'), u'type': u'broad_leaved', u'id': u'2405872915'}
{u'natural': u'tree', u'created': {u'changeset': u'35623297', u'version':

## Street check

In [177]:
print db.pa.distinct('address.street')

[u'School Drive', u'S Fraser Street', u'South Allen Street', u'South Pugh Street', u'West College Avenue', u'East College Avenue', u'East Calder Way', u'East Beaver Avenue', u'West Beaver Avenue', u'Food Science Building', u'Transfer Road', u'Premiere Drive', u'South Buckhout Street', u'N Patterson Street', u'South Atherton Street', u'Hiester Street', u'S Hiester Street', u'S Garner Street', u'S Sparks Street', u'North Atherton Street', u'McAllister Alley', u"McAlevy's Fort Road", u'Montauk Circle', u'South Burrowes Street', u'South Fraser Street', u'West Calder Way', u'Miller Alley', u'Northland Center', u'Benner Pike', u'E Main Street', u'S. Fraser Street', u'S Atherton Street', u'Houser Road', u'Earlystown Road', u'East Specht Street', u'US 522', u'Post Office Box', u'Pollock Road', u'W College Avenue', u'South Butz Street', u'Boalsburg Pike', u'West Park Avenue', u'Martin Street', u'West Aaron Drive', u'Blue Course Drive', u'Innovation Boulevard', u'John Wert Road', u'The 300 Build

## Ammenity

In [188]:
cursor = db.pa.aggregate([
    {'$group': {'_id': "$amenity", 'count': {'$sum': 1}}}])
    
for doc in cursor:
    pprint.pprint(doc)

#print db.pa.distinct('amenity')

{u'_id': u'fountain', u'count': 2}
{u'_id': None, u'count': 347278}
{u'_id': u'restaurant', u'count': 88}
{u'_id': u'place_of_worship', u'count': 96}
{u'_id': u'school', u'count': 171}
{u'_id': u'toilets', u'count': 6}
{u'_id': u'fuel', u'count': 20}
{u'_id': u'post_office', u'count': 8}
{u'_id': u'grave_yard', u'count': 102}
{u'_id': u'parking', u'count': 155}
{u'_id': u'hospital', u'count': 8}
{u'_id': u'parking_entrance', u'count': 2}
{u'_id': u'library', u'count': 3}
{u'_id': u'community_centre', u'count': 1}
{u'_id': u'pub', u'count': 15}
{u'_id': u'cafe', u'count': 13}
{u'_id': u'bank', u'count': 20}
{u'_id': u'arts_centre', u'count': 1}
{u'_id': u'post_box', u'count': 4}
{u'_id': u'drinking_water', u'count': 1}
{u'_id': u'food_court', u'count': 2}
{u'_id': u'recycling', u'count': 4}
{u'_id': u'fire_station', u'count': 8}
{u'_id': u'fast_food', u'count': 22}
{u'_id': u'shelter', u'count': 14}
{u'_id': u'public_building', u'count': 5}
{u'_id': u'cinema', u'count': 3}
{u'_id': u'th